In [ ]:
#LIBRERIAS
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta, datetime
import warnings
warnings.filterwarnings('ignore')

#CONFIGURACIÓN
plt.style.use("seaborn-v0_8")
sns.set_palette("Set2")

#TICKETS
tickers = ["META", "AAPL", "AMZN", "NFLX", "GOOGL"]

#ESTABLECER FECHA
fecha_final   = datetime.utcnow().date()
fecha_inicial = fecha_final - timedelta(252 * 10) #ULTIMOS 10 AÑOS

#DESCARGAR DATOS 
data = yf.download(tickers = tickers, start = fecha_inicial, end = fecha_final)["Close"]

In [ ]:
data.to_csv("data.csv", sep = ",", index = True)

# ANÁLISIS EXPLOTARIO

In [ ]:
#IMPRIMIR ENCABEZADOS
data.head()

In [ ]:
#IMPRIMIR DESCRIPCIÓN DE LAS ACCIONES
data.describe()

In [ ]:
#CREAR TIUTLO DEL GRAFICO
periodo = str(fecha_inicial.year) + "-" + str(fecha_final.year)
titulo = f"Precios Históricos Diarios de las Acciones ({periodo})"
data.plot(figsize=(15,6))
plt.title(titulo)
plt.ylabel("Precio (USD)")
plt.xlabel("Fecha")
plt.show()

# CALCULA RENDIMIENTOS DIARIOS

In [ ]:
#CALCULAR RENDIMIENTOS
rendimientos = data.pct_change().dropna()

#IMPRIMIR RENDIMIENTOS
rendimientos

In [ ]:
#GRAFICANDO EL DESEMPEÑO HISTÓRICO DE LAS ACCIONES
(1 + rendimientos).cumprod().plot(figsize = (15, 6))

In [ ]:
#CALCULAR LA VOLATILIDAD ANUAL DE LAS ACCIONES
volatilidad = rendimientos.std() * (252 ** 0.5)
print("Volatilidad Anualizada: \n", volatilidad)

In [ ]:
#CORRELACIÓN ENTRE ACCIONES
plt.figure(figsize = (15, 6))
sns.heatmap(rendimientos.corr(), annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Matriz de correlación de rendimientos")
plt.show()

# MÉTRICAS FINANCIERAS

## RATIO SHARPE

In [ ]:
tlr = 0.03 # TASA LIBRE DE RIESGO 3%
dif_rendi_diario = rendimientos - (tlr / 252) # RENDIMIENTOS VS TLR DIARIOS
sharpe = (dif_rendi_diario.mean() * 252) / (rendimientos.std() * (252 ** 0.5))
sharpe.sort_values(ascending=False).to_frame("Sharpe Anual")

## MÁXIMO DRAWDOWN

In [ ]:
## FUNCIÓN PARA CALCULAR EL MÁXIMO DRAWDOWN
def max_drawdown(df):
    cum  = (1 + df).cumprod()
    pico = cum.cummax()
    dd   = (cum / pico - 1)
    return dd.min()

In [ ]:
#CALCULAR EL MÁXIMO DRAWDOWN
drawdowns = rendimientos.apply(max_drawdown)
drawdowns.to_frame("Max Drawdown")

In [ ]:
#VISUALIZAR MAX DRAWDOWN
def graficar_drawdown(activo):
    cum_   = (1 + rendimientos[activo]).cumprod()
    peak_  = cum_.cummax()
    dd_    = cum_/peak_ - 1
    titulo = "Drawdown " + activo

    dd_.plot(figsize=(15,6))
    plt.title(titulo)
    plt.ylabel("Drawdown")
    plt.show()

In [ ]:
#SELECCIONAR UN ACTIVO ("META", "AAPL", "AMZN", "NFLX", "GOOGL")
graficar_drawdown("GOOGL")

## VaR (VALOR EN RIESGO)

In [ ]:
alfa = 0.05 # 5% de cola = 95% de confianza
VaR = rendimientos.quantile(alfa)
VaR.to_frame("VaR 95% diario")